In [16]:
import pandas as pd
from yelpapi import YelpAPI

In [77]:
df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_RAW.csv")

# 1. Start processing "easier" columns

In [78]:
#Process 'fee', clean and make it to an int
df['fee'] = df['fee'].str.rstrip(' kr/mån')
df["fee"] = df["fee"].str.replace(" ", "")
df["fee"] = pd.to_numeric(df["fee"] , errors='coerce')
df = df.dropna(subset=['fee'])
df['fee'] = df['fee'].astype('int')
#Fee succesfully 'cleaned' and data type corrected
 
#

In [80]:
df["location"].value_counts()

östermalm, stockholms kommun                         2105
vasastan, stockholms kommun                          1462
kungsholmen, stockholms kommun                       1236
södermalm, stockholms kommun                          916
råsunda, solna kommun                                 406
                                                     ... 
fredhäll / marieberg, stockholms kommun                 1
hornsbergs strand - kungshomen, stockholms kommun       1
kungsholmen/kungsholms strand, stockholms kommun        1
frösunda - ritorp, solna kommun                         1
vasastan, norrmalm, stockholms kommun                   1
Name: location, Length: 639, dtype: int64

# Yelp API calls

In [20]:
yelp_api = YelpAPI("KulP_1xAbhj4PcwcltixYR5hz4qMJ2aarTp4uNP_bBED4CsgP1nqY0bZrDxRMMSsZYwqvirOQ1Dy--6v3Y2yS4lBVPmfebDVdXGukr74OZEKRNoivTBiORBJ0v_iYXYx")
response = yelp_api.search_query(location=adress, radius=2000, limit=1)


KeyError: 1

In [26]:
latitude = response['region']['center']['latitude']
longitude = response['region']['center']['longitude']
pointsOfInterestNearby = response['total']





In [25]:
pointsOfInterestsNearby

240